# truck-n-trailer

<a href="https://colab.research.google.com/github/ericbreh/truck-n-trailer/blob/main/truck-n-trailer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import sys  #testdsfadfadsdsf
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install polytope
  !pip install -q pyomo
  !apt-get install -y -qq glpk-utils
  !apt-get install -y -qq coinor-cbc
  !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
  !unzip -o -q ipopt-linux64

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# Increase animation embed limit to avoid size warnings (set to 150 MB)
matplotlib.rcParams['animation.embed_limit'] = 350
import pyomo.environ as pyo
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

### Core Dynamics

- states: `[x, y, θ_truck, θ_trailer]`
- controls: `[v, φ]` (velocity and steering angle)

In [13]:
# --- Code 2: Dynamics ---
def truck_trailer_dynamics(q, u, L, d):
    # q = [x, y, theta_t, theta_l, v, phi]
    # u = [a, phi_dot]
    x, y, theta_t, theta_l, v, phi = q
    a, phi_dot = u

    x_dot = v * np.cos(theta_t)
    y_dot = v * np.sin(theta_t)
    theta_t_dot = (v / L) * np.tan(phi)
    theta_l_dot = (v / d) * np.sin(theta_t - theta_l)
    v_dot = a
    phi_dot = phi_dot

    return np.array([x_dot, y_dot, theta_t_dot, theta_l_dot, v_dot, phi_dot])


### Pyomo Model

In [14]:
# --- Code 3: Pyomo Model ---
def build_pyomo_model(params):
    N = params["N"]
    dt = params["dt"]
    L = params["L"]
    d = params["d"]
    target = params["target"]

    # Control bounds
    a_min = params.get("a_min", -2.0)
    a_max = params.get("a_max", 2.0)
    phi_dot_min = params.get("phi_dot_min", -1.0)
    phi_dot_max = params.get("phi_dot_max", 1.0)

    # State bounds
    v_min = params.get("v_min", 0.0)
    v_max = params.get("v_max", 3.0)
    phi_min = params.get("phi_min", -1.5)
    phi_max = params.get("phi_max", 1.5)

    w_control = params.get("w_control", 0.01)
    max_jackknife = params.get("max_jackknife_angle", np.pi/2)

    model = pyo.ConcreteModel()

    # Sets
    model.T = pyo.RangeSet(0, N)
    model.K = pyo.RangeSet(0, N - 1)
    model.S = pyo.RangeSet(0, 5)  # x, y, theta_t, theta_l, v, phi

    # State variables
    model.x = pyo.Var(model.S, model.T, domain=pyo.Reals)  

    # Control variables
    model.a = pyo.Var(model.K, bounds=(a_min, a_max))
    model.phi_dot = pyo.Var(model.K, bounds=(phi_dot_min, phi_dot_max))

    # Dynamics
    def dyn_rule(model, s, k):
        if s == 0:  # x
            return model.x[0, k+1] == model.x[0, k] + dt * (model.x[4, k] * pyo.cos(model.x[2, k]))
        if s == 1:  # y
            return model.x[1, k+1] == model.x[1, k] + dt * (model.x[4, k] * pyo.sin(model.x[2, k]))
        if s == 2:  # theta_t
            return model.x[2, k+1] == model.x[2, k] + dt * ((model.x[4, k] / L) * pyo.tan(model.x[5, k]))
        if s == 3:  # theta_l
            return model.x[3, k+1] == model.x[3, k] + dt * ((model.x[4, k] / d) * pyo.sin(model.x[2, k] - model.x[3, k]))
        if s == 4:  # v
            return model.x[4, k+1] == model.x[4, k] + dt * model.a[k]
        if s == 5:  # phi
            return model.x[5, k+1] == model.x[5, k] + dt * model.phi_dot[k]

    model.dyn = pyo.Constraint(model.S, model.K, rule=dyn_rule)

    # Jackknife constraint
    def jackknife_rule(model, t):
        return pyo.inequality(-max_jackknife, model.x[2, t] - model.x[3, t], max_jackknife)
    model.jackknife_cons = pyo.Constraint(model.T, rule=jackknife_rule)

    # Velocity & steering angle constraints
    model.vel_cons = pyo.Constraint(model.T, rule=lambda m, t: pyo.inequality(v_min, m.x[4, t], v_max))
    model.phi_cons = pyo.Constraint(model.T, rule=lambda m, t: pyo.inequality(phi_min, m.x[5, t], phi_max))

    # Terminal constraints (optional, for stability and feasibility)
    # Option 1: Zero terminal state constraint (hard constraint - ensures stability but may cause infeasibility)
    # Option 2: Terminal set constraint (constrain final state to be in a set around target)
    # Option 3: No terminal constraint (current approach - soft constraint via objective)
    
    use_terminal_constraint = params.get("use_terminal_constraint", False)
    terminal_tolerance = params.get("terminal_tolerance", 0.1)  # Tolerance for terminal set
    
    if use_terminal_constraint:
        terminal_type = params.get("terminal_constraint_type", "set")  # "zero", "set", or "none"
        
        if terminal_type == "zero":
            # Zero terminal state constraint: x[N] = target (hard constraint)
            # This ensures stability but may make problem infeasible
            def terminal_zero_rule(model, i):
                return model.x[i, N] == target[i]
            model.terminal_cons = pyo.Constraint(range(4), rule=terminal_zero_rule)
            
        elif terminal_type == "set":
            # Terminal set constraint: ||x[N] - target|| <= terminal_tolerance
            # This is a compromise: ensures final state is close to target while maintaining feasibility
            def terminal_set_rule(model):
                # Constraint: sum((x[i,N] - target[i])^2) <= terminal_tolerance^2
                return sum((model.x[i, N] - target[i])**2 for i in range(4)) <= terminal_tolerance**2
            model.terminal_cons = pyo.Constraint(rule=terminal_set_rule)

    # Objective
    def obj_rule(model):
        final_err = sum((model.x[i, N] - target[i])**2 for i in range(4))
        control_effort = sum(model.a[k]**2 + model.phi_dot[k]**2 for k in model.K)
        return final_err + w_control * control_effort
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    return model


MPC

In [15]:
# --- Code 4: Helpers ---
def set_initial_guess(model, u_guess):
    N = u_guess.shape[1]
    for k in range(N):
        model.a[k].set_value(float(u_guess[0, k]))
        model.phi_dot[k].set_value(float(u_guess[1, k]))

def extract_solution(model, params):
    N = params["N"]
    x_opt = np.zeros((6, N+1))
    u_opt = np.zeros((2, N))
    for i in range(6):
        for t in range(N+1):
            x_opt[i, t] = pyo.value(model.x[i, t])
    for k in range(N):
        u_opt[0, k] = pyo.value(model.a[k])
        u_opt[1, k] = pyo.value(model.phi_dot[k])
    return x_opt, u_opt

def run_mpc_loop(params):
    MAX_STEPS = params.get("max_steps", 150)
    TARGET_TOL = params.get("target_tol", 0.5)

    model = build_pyomo_model(params)
    solver = pyo.SolverFactory("ipopt")
    solver.options["max_iter"] = params.get("solver_max_iter", 500)
    solver.options["tol"] = params.get("solver_tol", 1e-6)
    solver.options["print_level"] = params.get("solver_print_level", 0)

    q_current = params["q0"].copy()
    N = params["N"]

    # initial guess for controls
    u_guess = np.zeros((2, N))
    u_guess[0, :] = params.get("a_init", 0.0)
    u_guess[1, :] = params.get("phi_dot_init", 0.0)

    # Fix initial state
    for i in range(6):
        model.x[i, 0].fix(float(q_current[i]))

    q_history = [q_current.copy()]
    u_history = []
    step = 0

    print("Starting MPC simulation...")

    while np.linalg.norm(q_current[:2] - params["target"][:2]) > TARGET_TOL and step < MAX_STEPS:
        for i in range(6):
            model.x[i, 0].fix(float(q_current[i]))

        set_initial_guess(model, u_guess)
        result = solver.solve(model, tee=False)
        term = result.solver.termination_condition
        if term not in (pyo.TerminationCondition.optimal, pyo.TerminationCondition.locallyOptimal):
            print(f"Solver warning: {term} at step {step}")
            break

        x_opt, u_opt = extract_solution(model, params)
        u_star = u_opt[:, 0]
        q_dot = truck_trailer_dynamics(q_current, u_star, params["L"], params["d"])
        q_next = q_current + params["dt"] * q_dot

        q_history.append(q_next.copy())
        u_history.append(u_star.copy())

        q_current = q_next.copy()
        step += 1

        u_guess = np.hstack([u_opt[:, 1:], u_opt[:, -1:]])

    print(f"MPC finished in {step} steps. Final error: {np.linalg.norm(q_current[:2] - params['target'][:2]):.3f} m")
    return np.array(q_history), np.array(u_history)


### Simulation

In [16]:
# --- Code 5: Plots & Animation ---
def plot_trajectory(q_hist, u_hist, params):
    q_hist = np.atleast_2d(q_hist)
    if q_hist.shape[0] == 6:
        q_hist = q_hist.T

    fig, axes = plt.subplots(3, 2, figsize=(14, 12))

    # States
    state_labels = ["x (m)", "y (m)", "theta_t (rad)", "theta_l (rad)", "v (m/s)", "phi (rad)"]
    for i, ax in enumerate(axes.flatten()):
        if i < 6:
            ax.plot(np.arange(q_hist.shape[0])*params["dt"], q_hist[:, i], "-o", markersize=3)
            ax.set_title(state_labels[i])
            ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Controls
    if len(u_hist) > 0:
        t = np.arange(len(u_hist))*params["dt"]
        plt.figure(figsize=(10,4))
        plt.step(t, u_hist[:, 0], where="post", label="a (m/s²)")
        plt.step(t, u_hist[:, 1], where="post", label="phi_dot (rad/s)")
        plt.xlabel("Time (s)")
        plt.ylabel("Control")
        plt.title("Applied Controls")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

def draw_truck_trailer_2d(ax, x, y, theta_t, theta_l, L, d, alpha=1.0, patches_list=None):
    """
    Draw a 2D representation of truck and trailer with proper shapes.
    x, y: hitch position (rear axle of truck)
    theta_t: truck heading angle
    theta_l: trailer heading angle
    L: truck wheelbase
    d: trailer length
    patches_list: list to append patches to (for cleanup in animation)
    """
    from matplotlib.patches import Rectangle, Circle, Polygon, FancyArrowPatch
    
    if patches_list is None:
        patches_list = []
    
    # Truck: from rear axle (hitch) to front axle
    truck_rear = np.array([x, y])
    truck_front = np.array([x + L*np.cos(theta_t), y + L*np.sin(theta_t)])
    
    # Trailer: from hitch to rear axle
    trailer_rear = np.array([x - d*np.cos(theta_l), y - d*np.sin(theta_l)])
    trailer_hitch = np.array([x, y])
    
    # Draw truck body (rectangle using Polygon for rotation)
    truck_center = (truck_rear + truck_front) / 2
    truck_width = 0.8
    truck_length = L
    # Create rectangle corners in local coordinates
    corners_local = np.array([
        [-truck_length/2, -truck_width/2],
        [truck_length/2, -truck_width/2],
        [truck_length/2, truck_width/2],
        [-truck_length/2, truck_width/2]
    ])
    # Rotate and translate
    rot_matrix = np.array([[np.cos(theta_t), -np.sin(theta_t)],
                           [np.sin(theta_t), np.cos(theta_t)]])
    corners_global = (rot_matrix @ corners_local.T).T + truck_center
    truck_poly = Polygon(corners_global, facecolor='lightblue', edgecolor='blue', 
                         linewidth=2, alpha=alpha, zorder=3)
    ax.add_patch(truck_poly)
    patches_list.append(truck_poly)
    
    # Draw trailer body (rectangle)
    trailer_center = (trailer_hitch + trailer_rear) / 2
    trailer_width = 0.9
    trailer_length = d
    corners_local = np.array([
        [-trailer_length/2, -trailer_width/2],
        [trailer_length/2, -trailer_width/2],
        [trailer_length/2, trailer_width/2],
        [-trailer_length/2, trailer_width/2]
    ])
    rot_matrix = np.array([[np.cos(theta_l), -np.sin(theta_l)],
                           [np.sin(theta_l), np.cos(theta_l)]])
    corners_global = (rot_matrix @ corners_local.T).T + trailer_center
    trailer_poly = Polygon(corners_global, facecolor='lightcoral', edgecolor='red', 
                          linewidth=2, alpha=alpha, zorder=2)
    ax.add_patch(trailer_poly)
    patches_list.append(trailer_poly)
    
    # Draw wheels
    wheel_radius = 0.15
    wheel_offset = 0.25
    # Truck front wheels
    for side in [-1, 1]:
        wheel_center = truck_front + side * wheel_offset * np.array([-np.sin(theta_t), np.cos(theta_t)])
        wheel = Circle(wheel_center, wheel_radius, facecolor='black', edgecolor='darkgray', 
                      linewidth=1, zorder=4, alpha=alpha)
        ax.add_patch(wheel)
        patches_list.append(wheel)
    
    # Truck rear wheels (at hitch)
    for side in [-1, 1]:
        wheel_center = truck_rear + side * wheel_offset * np.array([-np.sin(theta_t), np.cos(theta_t)])
        wheel = Circle(wheel_center, wheel_radius, facecolor='black', edgecolor='darkgray', 
                      linewidth=1, zorder=4, alpha=alpha)
        ax.add_patch(wheel)
        patches_list.append(wheel)
    
    # Trailer wheels
    for side in [-1, 1]:
        wheel_center = trailer_rear + side * wheel_offset * np.array([-np.sin(theta_l), np.cos(theta_l)])
        wheel = Circle(wheel_center, wheel_radius, facecolor='black', edgecolor='darkgray', 
                      linewidth=1, zorder=4, alpha=alpha)
        ax.add_patch(wheel)
        patches_list.append(wheel)
    
    # Draw hitch point
    hitch_circle = Circle((x, y), 0.1, facecolor='black', edgecolor='white', 
                         linewidth=1, zorder=5, alpha=alpha)
    ax.add_patch(hitch_circle)
    patches_list.append(hitch_circle)
    
    # Draw direction arrows using FancyArrowPatch for proper removal
    arrow_length = 0.5
    # Truck direction
    arrow_t = FancyArrowPatch((x, y), 
                              (x + arrow_length*np.cos(theta_t), y + arrow_length*np.sin(theta_t)),
                              arrowstyle='->', mutation_scale=15, linewidth=2,
                              color='blue', alpha=alpha, zorder=6)
    ax.add_patch(arrow_t)
    patches_list.append(arrow_t)
    # Trailer direction
    arrow_l = FancyArrowPatch((x, y),
                              (x - arrow_length*np.cos(theta_l), y - arrow_length*np.sin(theta_l)),
                              arrowstyle='->', mutation_scale=15, linewidth=2,
                              color='red', alpha=alpha, zorder=6)
    ax.add_patch(arrow_l)
    patches_list.append(arrow_l)
    
    return patches_list

def animate_trajectory_with_obstacles(q_hist, params, interval=80, max_frames=500):
    """
    Animate truck-trailer trajectory with moving obstacles.
    """
    q_hist = np.atleast_2d(q_hist)
    if q_hist.shape[0] == 6:
        q_hist = q_hist.T

    L = params["L"]
    d = params["d"]
    M_original = len(q_hist)
    
    # Downsample if too many frames
    if max_frames is not None and M_original > max_frames:
        frame_step = M_original // max_frames
        q_hist = q_hist[::frame_step]
        M = len(q_hist)
        print(f"Downsampling animation: {M_original} frames -> {M} frames (step={frame_step})")
    else:
        frame_step = 1
        M = M_original
        if max_frames is None and M_original > 500:
            print(f"Animation has {M_original} frames (no downsampling). Consider using max_frames parameter if animation is too large.")

    fig, ax = plt.subplots(figsize=(14, 12))
    ax.set_aspect("equal")
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlabel("x (m)", fontsize=12)
    ax.set_ylabel("y (m)", fontsize=12)
    ax.set_title("Truck-Trailer Navigation with Moving Obstacles", fontsize=14, fontweight='bold')

    # Calculate limits including target and obstacles
    all_x = [q_hist[:,0], [params["target"][0]]]
    all_y = [q_hist[:,1], [params["target"][1]]]
    
    # Include obstacle positions in limits
    if "obstacle_history" in params:
        for obs_hist in params["obstacle_history"]:
            obs_positions = np.array(obs_hist["positions"])
            all_x.append(obs_positions[:, 0])
            all_y.append(obs_positions[:, 1])
    
    all_x = np.concatenate(all_x)
    all_y = np.concatenate(all_y)
    margin = max(8.0, 0.25*max(np.ptp(all_x), np.ptp(all_y)))
    ax.set_xlim(all_x.min()-margin, all_x.max()+margin)
    ax.set_ylim(all_y.min()-margin, all_y.max()+margin)
    
    # Draw full path
    path_line, = ax.plot(q_hist[:,0], q_hist[:,1], "b--", alpha=0.3, lw=2, label="Truck Path", zorder=1)
    
    # Draw start and target
    q0 = params["q0"]
    ax.plot(q0[0], q0[1], "g*", ms=20, mew=2, label="Start", zorder=10)
    target_x, target_y = params["target"][0], params["target"][1]
    ax.plot(target_x, target_y, "r*", ms=20, mew=2, label="Target", zorder=10)
    
    # Draw obstacle paths (trajectories)
    if "obstacle_history" in params:
        for i, obs_hist in enumerate(params["obstacle_history"]):
            obs_positions = np.array(obs_hist["positions"])
            if len(obs_positions) > 1:
                ax.plot(obs_positions[:, 0], obs_positions[:, 1], "r--", 
                       linewidth=1.5, alpha=0.4, label=f"Obstacle {i+1} Path", zorder=1)
    
    # Initialize patches list
    patches_list = []
    
    title_text = ax.text(0.5, 0.98, "", ha="center", va="top", transform=ax.transAxes, 
                        fontsize=12, fontweight='bold', 
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    info_text = ax.text(0.02, 0.98, "", ha="left", va="top", transform=ax.transAxes,
                       fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

    def init():
        title_text.set_text("")
        info_text.set_text("")
        return [title_text, info_text]

    def update(frame):
        # Clear previous patches
        for patch in patches_list:
            try:
                patch.remove()
            except:
                pass
        patches_list.clear()
        
        # Current truck state
        actual_frame = frame * frame_step
        if actual_frame >= len(q_hist):
            actual_frame = len(q_hist) - 1
        
        x, y, theta_t, theta_l = q_hist[actual_frame, :4]
        v = q_hist[actual_frame, 4] if q_hist.shape[1] > 4 else 0.0
        
        # Draw truck-trailer
        draw_truck_trailer_2d(ax, x, y, theta_t, theta_l, L, d, alpha=1.0, patches_list=patches_list)
        
        # Draw obstacles at current time step
        from matplotlib.patches import Circle
        if "obstacle_history" in params:
            for i, obs_hist in enumerate(params["obstacle_history"]):
                # Get obstacle position at current frame
                if actual_frame < len(obs_hist["positions"]):
                    obs_pos = obs_hist["positions"][actual_frame]
                else:
                    # Extrapolate if beyond tracked history
                    last_pos = obs_hist["positions"][-1]
                    dt = params["dt"]
                    steps_ahead = actual_frame - len(obs_hist["positions"]) + 1
                    obs_pos = [
                        last_pos[0] + obs_hist["velocity"][0] * steps_ahead * dt,
                        last_pos[1] + obs_hist["velocity"][1] * steps_ahead * dt
                    ]
                
                # Draw obstacle
                obs_circle = Circle(obs_pos, obs_hist["radius"], 
                                  facecolor='red', edgecolor='darkred', 
                                  linewidth=2, alpha=0.8, zorder=6)
                ax.add_patch(obs_circle)
                patches_list.append(obs_circle)
                
                # Draw safety zone
                safety_radius = obs_hist["radius"] + params.get("truck_radius", 1.5) + params.get("safety_margin", 2.0)
                safety_circle = Circle(obs_pos, safety_radius, 
                                     facecolor='none', edgecolor='orange', 
                                     linewidth=1, linestyle='--', alpha=0.3, zorder=1)
                ax.add_patch(safety_circle)
                patches_list.append(safety_circle)
        
        # Update text
        time_val = actual_frame * params["dt"]
        dist_to_target = np.linalg.norm([x - target_x, y - target_y])
        title_text.set_text(f"Step {actual_frame+1}/{M_original} | Time: {time_val:.1f}s")
        info_text.set_text(f"Position: ({x:.2f}, {y:.2f})\n"
                          f"Truck θ: {np.degrees(theta_t):.1f}°\n"
                          f"Trailer θ: {np.degrees(theta_l):.1f}°\n"
                          f"Distance to target: {dist_to_target:.2f} m\n"
                          f"Velocity: {v:.2f} m/s")
        
        return [title_text, info_text] + patches_list

    anim = FuncAnimation(fig, update, init_func=init, frames=M, interval=interval, blit=False, repeat=True)
    ax.legend(loc="upper right", fontsize=10)
    return anim

def animate_trajectory(q_hist, params, interval=80, max_frames=500):
    """
    Improved 2D animation with proper truck-trailer visualization.
    
    Parameters:
    -----------
    q_hist : array
        State history
    params : dict
        Parameters dictionary
    interval : int
        Animation interval in milliseconds
    max_frames : int
        Maximum number of frames to animate (downsample if needed)
    """
    q_hist = np.atleast_2d(q_hist)
    if q_hist.shape[0] == 6:
        q_hist = q_hist.T

    L = params["L"]
    d = params["d"]
    M_original = len(q_hist)
    
    # Downsample if too many frames to keep animation size manageable
    if max_frames is not None and M_original > max_frames:
        frame_step = M_original // max_frames
        q_hist = q_hist[::frame_step]
        M = len(q_hist)
        print(f"Downsampling animation: {M_original} frames -> {M} frames (step={frame_step})")
    else:
        frame_step = 1
        M = M_original
        if max_frames is None and M_original > 500:
            print(f"Animation has {M_original} frames (no downsampling). Consider using max_frames parameter if animation is too large.")

    fig, ax = plt.subplots(figsize=(12, 12))
    ax.set_aspect("equal")
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlabel("x (m)", fontsize=12)
    ax.set_ylabel("y (m)", fontsize=12)
    ax.set_title("Truck-Trailer MPC Navigation", fontsize=14, fontweight='bold')

    # Calculate proper limits including target
    all_x = np.concatenate([q_hist[:,0], [params["target"][0]]])
    all_y = np.concatenate([q_hist[:,1], [params["target"][1]]])
    margin = max(8.0, 0.25*max(np.ptp(all_x), np.ptp(all_y)))
    ax.set_xlim(all_x.min()-margin, all_x.max()+margin)
    ax.set_ylim(all_y.min()-margin, all_y.max()+margin)
    
    # Draw full path (hitch trajectory)
    path_line, = ax.plot(q_hist[:,0], q_hist[:,1], "k--", alpha=0.3, lw=1.5, label="Path (hitch)", zorder=1)
    
    # Draw initial position (beginning) as green star
    q0 = params["q0"]
    initial_x, initial_y = q0[0], q0[1]
    initial_marker = ax.plot(initial_x, initial_y, "g*", ms=20, mew=2, label="Start", zorder=10)[0]
    
    # Draw target1 if provided (for combined forward/reverse navigation)
    if "target1" in params:
        target1_x, target1_y = params["target1"][0], params["target1"][1]
        target1_marker = ax.plot(target1_x, target1_y, "mo", ms=18, mew=2, label="Target 1", zorder=10)[0]
    
    # Draw target with orientation if available
    target_x, target_y = params["target"][0], params["target"][1]
    target_marker = ax.plot(target_x, target_y, "r*", ms=20, mew=2, label="Target", zorder=10)[0]
    
    # Draw target orientation if provided
    if len(params["target"]) >= 4:
        target_theta_t = params["target"][2]
        target_theta_l = params["target"][3]
        # Draw target truck orientation
        ax.arrow(target_x, target_y, 1.5*np.cos(target_theta_t), 1.5*np.sin(target_theta_t),
                head_width=0.3, head_length=0.2, fc='green', ec='green', alpha=0.5, linestyle='--', zorder=9)
        # Draw target trailer orientation
        ax.arrow(target_x, target_y, -1.5*np.cos(target_theta_l), -1.5*np.sin(target_theta_l),
                head_width=0.3, head_length=0.2, fc='orange', ec='orange', alpha=0.5, linestyle='--', zorder=9)
    
    # Initialize patches list for animation
    patches_list = []
    
    title_text = ax.text(0.5, 0.98, "", ha="center", va="top", transform=ax.transAxes, 
                        fontsize=12, fontweight='bold', 
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    info_text = ax.text(0.02, 0.98, "", ha="left", va="top", transform=ax.transAxes,
                       fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

    def init():
        title_text.set_text("")
        info_text.set_text("")
        return [title_text, info_text]

    def update(frame):
        # Clear previous frame's patches
        for patch in patches_list:
            try:
                patch.remove()
            except:
                pass
        patches_list.clear()
        
        x, y, theta_t, theta_l = q_hist[frame, :4]
        v = q_hist[frame, 4] if q_hist.shape[1] > 4 else 0.0
        phi = q_hist[frame, 5] if q_hist.shape[1] > 5 else 0.0
        
        # Draw current truck-trailer (patches_list will be populated)
        draw_truck_trailer_2d(ax, x, y, theta_t, theta_l, L, d, alpha=1.0, patches_list=patches_list)
        
        # Update text
        time_val = frame * frame_step * params["dt"]
        dist_to_target = np.linalg.norm([x - target_x, y - target_y])
        title_text.set_text(f"Step {frame*frame_step+1}/{M_original} | Time: {time_val:.1f}s")
        info_text.set_text(f"Position: ({x:.2f}, {y:.2f})\n"
                          f"Truck θ: {np.degrees(theta_t):.1f}°\n"
                          f"Trailer θ: {np.degrees(theta_l):.1f}°\n"
                          f"Distance to target: {dist_to_target:.2f} m\n"
                          f"Velocity: {v:.2f} m/s\n"
                          f"Steering: {np.degrees(phi):.1f}°")
        
        return [title_text, info_text]

    anim = FuncAnimation(fig, update, init_func=init, frames=M, interval=interval, blit=False, repeat=True)
    ax.legend(loc="upper right", fontsize=10)
    return anim


### Forward Point-to-Point Navigation

In [17]:
params_forward_point = {
    "L": 2.0, "d": 5.0,
    "target": np.array([np.random.uniform(0,15), np.random.uniform(-15,15), 0.0, 0.0]),
    "q0": np.array([0.0,0.0,0.0,0.0,0.0,0.0]),
    "dt": 0.1, "N": 10,
    "v_min": 0.0, "v_max": 3.0, "phi_min": -1.5, "phi_max": 1.5,
    "a_min": -2.0, "a_max": 2.0, "phi_dot_min": -1.0, "phi_dot_max": 1.0,
    "max_jackknife_angle": np.pi/2,
    "w_control": 0.01,
    "max_steps": 200, "target_tol": 0.5,
    "solver_max_iter": 500, "solver_tol": 1e-6, "solver_print_level": 0,
    "a_init": 0.1, "phi_dot_init": 0.0
}

q_hist_fwd, u_hist_fwd = run_mpc_loop(params_forward_point)
plot_trajectory(q_hist_fwd, u_hist_fwd, params_forward_point)

anim_fwd = animate_trajectory(q_hist_fwd, params_forward_point, interval=80)
display(HTML(anim_fwd.to_jshtml()))

In [19]:
# Define target - closer to make it more achievable
target_with_obstacles = np.array([18.0, 6.0, 0.0, 0.0])

# Create multiple obstacles moving across the path
# Position obstacles to cross at different times, allowing gaps for the truck to pass
# Obstacle 1: Crosses early in the path (from left to right, but offset so truck can go around)
obstacle1 = {
    "position": [4.0, 1.5],  # Lower y position, truck can go above
    "velocity": [1.5, 0.0],  # Moving right at 1.5 m/s (slower)
    "radius": 0.2,
    "initial_position": [4.0, 1.5],
}

# Obstacle 2: Crosses in the middle (from top to bottom, offset to side)
obstacle2 = {
    "position": [12.0, 10.0],  # Higher start, crosses later
    "velocity": [0.0, -2.0],  # Moving down at 2.0 m/s
    "radius": 0.2,
    "initial_position": [12.0, 10.0],
}

# Obstacle 3: Crosses later (from right to left, but offset)
obstacle3 = {
    "position": [16.0, 3.0],  # Lower position, truck can go above
    "velocity": [-1.5, 0.0],  # Moving left at 1.5 m/s
    "radius": 0.2,
    "initial_position": [16.0, 3.0],
}

params_obstacles_demo = {
    "L": 2.0, "d": 5.0,
    "target": target_with_obstacles,
    "q0": np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    "dt": 0.1, "N": 30,  # Longer horizon for better obstacle avoidance
    "v_min": 0.0, "v_max": 3.0, 
    "phi_min": -1.5, "phi_max": 1.5,
    "a_min": -2.0, "a_max": 2.0, 
    "phi_dot_min": -1.0, "phi_dot_max": 1.0,
    "max_jackknife_angle": np.pi/2,
    "w_control": 0.01,
    "max_steps": 400, "target_tol": 0.5,
    "solver_max_iter": 2000, "solver_tol": 1e-5, "solver_print_level": 0,  # More iterations, relaxed tolerance
    "a_init": 0.1, "phi_dot_init": 0.0,
    
    # Multiple obstacles
    "obstacles": [obstacle1, obstacle2, obstacle3],
    "safety_margin": 1.2,  # Reduced safety margin to make problem more feasible
    "truck_radius": 1.5,    # Truck-trailer footprint radius (m)
}

# Run MPC with obstacle avoidance
print("=" * 60)
print("Forward Navigation with Multiple Moving Obstacles")
print("=" * 60)
print(f"Target: ({target_with_obstacles[0]:.1f}, {target_with_obstacles[1]:.1f})")
print(f"Number of obstacles: {len(params_obstacles_demo['obstacles'])}")
print()

q_hist_obstacles, u_hist_obstacles = run_mpc_loop_obstacles(params_obstacles_demo)

# Create visualization
fig, ax = plt.subplots(figsize=(16, 12))
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)

# Plot truck trajectory
ax.plot(q_hist_obstacles[:, 0], q_hist_obstacles[:, 1], "b-", linewidth=3, 
        label="Truck-Trailer Path", zorder=2, alpha=0.8)

# Plot start and target
ax.plot(params_obstacles_demo["q0"][0], params_obstacles_demo["q0"][1], 
        "g*", ms=25, mew=2, label="Start", zorder=10)
ax.plot(target_with_obstacles[0], target_with_obstacles[1], 
        "r*", ms=25, mew=2, label="Target", zorder=10)

# Plot obstacle trajectories and positions
from matplotlib.patches import Circle
colors = ['red', 'orange', 'purple']
if "obstacle_history" in params_obstacles_demo:
    for i, obs_hist in enumerate(params_obstacles_demo["obstacle_history"]):
        obs_positions = np.array(obs_hist["positions"])
        if len(obs_positions) > 1:
            # Draw obstacle path
            ax.plot(obs_positions[:, 0], obs_positions[:, 1], 
                   "--", linewidth=2, alpha=0.5, color=colors[i % len(colors)],
                   label=f"Obstacle {i+1} Path", zorder=1)
            
            # Draw initial and final positions
            circle_init = Circle(obs_positions[0], obs_hist["radius"], 
                               facecolor=colors[i % len(colors)], 
                               edgecolor='darkred', linewidth=2, 
                               alpha=0.7, zorder=5)
            circle_final = Circle(obs_positions[-1], obs_hist["radius"], 
                                facecolor=colors[i % len(colors)], 
                                edgecolor='black', linewidth=2, 
                                alpha=0.9, zorder=5)
            ax.add_patch(circle_init)
            ax.add_patch(circle_final)

ax.set_xlabel("x (m)", fontsize=14)
ax.set_ylabel("y (m)", fontsize=14)
ax.set_title("Truck-Trailer Navigation with Multiple Moving Obstacles", 
            fontsize=16, fontweight='bold')
ax.legend(loc='best', fontsize=11, framealpha=0.9)
plt.tight_layout()
plt.show()

# Create animation with moving obstacles
print("\n" + "=" * 60)
print("Generating animation with moving obstacles...")
print("=" * 60)

if 'animate_trajectory_with_obstacles' in globals():
    anim_obstacles = animate_trajectory_with_obstacles(
        q_hist_obstacles, params_obstacles_demo, interval=80, max_frames=600
    )
    display(HTML(anim_obstacles.to_jshtml()))
else:
    print("⚠️  Warning: animate_trajectory_with_obstacles not found.")
    print("   Please run Cell 11 (Plots & Animation) first to define the function.")
    print("   Using regular animation (obstacles won't move)...")
    anim_obstacles = animate_trajectory(
        q_hist_obstacles, params_obstacles_demo, interval=80, max_frames=600
    )
    display(HTML(anim_obstacles.to_jshtml()))


In [2]:
# THIS CELL explains how MPC obstacle avoidance works. 
import matplotlib.pyplot as plt
import numpy as np

# Simulate obstacle prediction over MPC horizon
dt = 0.1
N = 20  # Prediction horizon
horizon_time = N * dt  # 2.0 seconds

# Example obstacle
obstacle = {
    "position": [5.0, 8.0],
    "velocity": [0.0, -2.5],  # Moving down
    "radius": 0.3
}

# Predict obstacle positions over horizon
obstacle_trajectory = []
for t in range(N + 1):
    time = t * dt
    obs_x = obstacle["position"][0] + obstacle["velocity"][0] * time
    obs_y = obstacle["position"][1] + obstacle["velocity"][1] * time
    obstacle_trajectory.append([obs_x, obs_y])

obstacle_trajectory = np.array(obstacle_trajectory)

# Visualize prediction
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Left plot: Obstacle prediction over time
ax1.set_aspect("equal")
ax1.grid(True, alpha=0.3)
ax1.set_xlabel("x (m)", fontsize=12)
ax1.set_ylabel("y (m)", fontsize=12)
ax1.set_title("MPC Obstacle Prediction\n(Predicted positions over horizon)", fontsize=14, fontweight='bold')

# Plot predicted obstacle positions
for i, pos in enumerate(obstacle_trajectory):
    alpha = 0.3 + 0.7 * (i / len(obstacle_trajectory))  # Fade from past to future
    circle = plt.Circle(pos, obstacle["radius"], color='red', alpha=alpha, zorder=5)
    ax1.add_patch(circle)
    if i % 5 == 0:  # Label every 5th step
        ax1.text(pos[0] + 0.5, pos[1], f't={i*dt:.1f}s', fontsize=8, alpha=0.7)

# Draw trajectory line
ax1.plot(obstacle_trajectory[:, 0], obstacle_trajectory[:, 1], 'r--', 
         linewidth=2, alpha=0.5, label='Predicted Path', zorder=1)

# Mark initial position
ax1.plot(obstacle_trajectory[0, 0], obstacle_trajectory[0, 1], 'ro', 
         ms=10, label='Initial Position', zorder=10)
ax1.plot(obstacle_trajectory[-1, 0], obstacle_trajectory[-1, 1], 'r*', 
         ms=15, label=f'Predicted at t={horizon_time:.1f}s', zorder=10)

ax1.legend(loc='best')
ax1.set_xlim(4, 6)
ax1.set_ylim(2, 9)

# Right plot: How MPC uses this prediction
ax2.set_aspect("equal")
ax2.grid(True, alpha=0.3)
ax2.set_xlabel("x (m)", fontsize=12)
ax2.set_ylabel("y (m)", fontsize=12)
ax2.set_title("MPC Planning with Obstacle Avoidance\n(At each step, MPC plans around predicted positions)", 
              fontsize=14, fontweight='bold')

# Show truck starting position
truck_start = np.array([0.0, 0.0])
target = np.array([10.0, 5.0])

ax2.plot(truck_start[0], truck_start[1], 'g*', ms=20, label='Truck Start', zorder=10)
ax2.plot(target[0], target[1], 'b*', ms=20, label='Target', zorder=10)

# Show obstacle prediction at current time
for i in range(0, N+1, 3):  # Show every 3rd prediction
    pos = obstacle_trajectory[i]
    safety_radius = obstacle["radius"] + 1.5 + 1.2  # obstacle + truck + margin
    circle = plt.Circle(pos, safety_radius, color='orange', alpha=0.2, 
                       linestyle='--', linewidth=1, zorder=1)
    ax2.add_patch(circle)
    # Obstacle itself
    obs_circle = plt.Circle(pos, obstacle["radius"], color='red', alpha=0.4, zorder=5)
    ax2.add_patch(obs_circle)

ax2.plot(obstacle_trajectory[:, 0], obstacle_trajectory[:, 1], 'r--', 
         linewidth=2, alpha=0.5, label='Obstacle Prediction', zorder=1)

# Draw example safe path (what MPC would plan)
safe_path_x = [0, 2, 4, 6, 8, 10]
safe_path_y = [0, 1, 2.5, 4, 4.5, 5]
ax2.plot(safe_path_x, safe_path_y, 'b-', linewidth=3, alpha=0.7, 
         label='Example Safe Path (MPC Solution)', zorder=2)

ax2.legend(loc='best', fontsize=10)
ax2.set_xlim(-1, 11)
ax2.set_ylim(-1, 9)

plt.tight_layout()
plt.show()

print("=" * 70)
print("HOW MPC OBSTACLE AVOIDANCE WORKS")
print("=" * 70)
print(f"\n1. PREDICTION:")
print(f"   - MPC predicts obstacle positions for next {N} steps ({horizon_time:.1f} seconds)")
print(f"   - Uses constant velocity model: pos(t) = pos(0) + velocity × t")
print(f"   - At each step, obstacle position is: {obstacle['position']} + {obstacle['velocity']} × t")

print(f"\n2. CONSTRAINT GENERATION:")
print(f"   - For each time step t in horizon [0, {N}]:")
print(f"     * Predict obstacle position at time t")
print(f"     * Add constraint: ||truck_pos(t) - obstacle_pos(t)|| >= safety_radius")
print(f"   - Safety radius = obstacle_radius + truck_radius + margin")
print(f"   - Safety radius = {obstacle['radius']:.1f} + 1.5 + 1.2 = {obstacle['radius'] + 1.5 + 1.2:.1f} m")

print(f"\n3. OPTIMIZATION:")
print(f"   - MPC solves optimization problem with all predicted constraints")
print(f"   - Finds trajectory that:")
print(f"     * Reaches target (or gets close)")
print(f"     * Avoids all predicted obstacle positions")
print(f"     * Minimizes control effort")

print(f"\n4. REPLANNING:")
print(f"   - After executing first step, MPC:")
print(f"     * Updates obstacle positions (they've moved)")
print(f"     * Rebuilds prediction with new positions")
print(f"     * Replans entire trajectory")
print(f"   - This handles: model errors, disturbances, measurement noise")

print(f"\n5. WHY PREDICTABILITY IS GOOD:")
print(f"   - Predictable obstacles = better planning")
print(f"   - MPC can plan optimal paths when it knows obstacle trajectories")
print(f"   - Replanning handles any deviations from predictions")
print(f"   - For unpredictable obstacles, would need probabilistic/robust MPC")

print(f"\n6. CONSTRAINT EQUATION (from code):")
print(f"   For obstacle o and time step t:")
print(f"   obs_x(t) = obs_x0 + obs_vx × t × dt")
print(f"   obs_y(t) = obs_y0 + obs_vy × t × dt")
print(f"   ")
print(f"   Constraint: (truck_x(t) - obs_x(t))² + (truck_y(t) - obs_y(t))² >= min_distance²")
print(f"   ")
print(f"   This constraint is added for:")
print(f"   - Each obstacle (o = 0, 1, 2, ...)")
print(f"   - Each time step in horizon (t = 0, 1, 2, ..., {N})")
print(f"   - Total constraints: {len([obstacle])} obstacle × {N+1} time steps = {N+1} constraints")

print(f"\n7. WHY PREDICTABILITY IS EXPECTED:")
print(f"   - MPC is a PREDICTIVE controller - it uses predictions!")
print(f"   - If obstacles are predictable, MPC can plan optimal paths")
print(f"   - Replanning at each step handles:")
print(f"     * Model errors (truck doesn't follow exact predicted path)")
print(f"     * Disturbances (wind, road conditions)")
print(f"     * Measurement errors")
print(f"     * Small changes in obstacle behavior")
print(f"   - For truly unpredictable obstacles, would need:")
print(f"     * Probabilistic MPC (uncertainty in obstacle motion)")
print(f"     * Robust MPC (worst-case planning)")
print(f"     * Sensor fusion (tracking with uncertainty)")

print(f"\n8. CURRENT LIMITATIONS:")
print(f"   - Assumes constant velocity (linear prediction)")
print(f"   - No uncertainty handling (deterministic)")
print(f"   - No reactive obstacle behavior")
print(f"   - For real-world: would need sensor fusion, uncertainty bounds")
print("=" * 70)


In [ ]:
# --- MPC Model with Obstacle Avoidance ---
def build_pyomo_model_obstacles(params):
    """
    Build Pyomo model with obstacle avoidance constraints.
    
    Obstacles are defined in params["obstacles"] as a list of dictionaries:
    obstacles = [
        {
            "position": [x, y],  # Initial position
            "velocity": [vx, vy],  # Constant velocity (m/s)
            "radius": r,  # Obstacle radius (m)
        },
        ...
    ]
    """
    N = params["N"]
    dt = params["dt"]
    L = params["L"]
    d = params["d"]
    target = params["target"]

    # Control bounds
    a_min = params.get("a_min", -2.0)
    a_max = params.get("a_max", 2.0)
    phi_dot_min = params.get("phi_dot_min", -1.0)
    phi_dot_max = params.get("phi_dot_max", 1.0)

    # State bounds
    v_min = params.get("v_min", 0.0)
    v_max = params.get("v_max", 3.0)
    phi_min = params.get("phi_min", -1.5)
    phi_max = params.get("phi_max", 1.5)

    w_control = params.get("w_control", 0.01)
    max_jackknife = params.get("max_jackknife_angle", np.pi/2)
    
    # Obstacle parameters
    obstacles = params.get("obstacles", [])
    safety_margin = params.get("safety_margin", 2.0)  # Additional safety margin beyond obstacle radius
    truck_radius = params.get("truck_radius", 1.5)  # Approximate truck-trailer footprint radius

    model = pyo.ConcreteModel()

    # Sets
    model.T = pyo.RangeSet(0, N)
    model.K = pyo.RangeSet(0, N - 1)
    model.S = pyo.RangeSet(0, 5)  # x, y, theta_t, theta_l, v, phi
    model.O = pyo.RangeSet(0, len(obstacles) - 1) if obstacles else pyo.RangeSet(0, 0)

    # State variables
    model.x = pyo.Var(model.S, model.T, domain=pyo.Reals)  

    # Control variables
    model.a = pyo.Var(model.K, bounds=(a_min, a_max))
    model.phi_dot = pyo.Var(model.K, bounds=(phi_dot_min, phi_dot_max))

    # Dynamics
    def dyn_rule(model, s, k):
        if s == 0:  # x
            return model.x[0, k+1] == model.x[0, k] + dt * (model.x[4, k] * pyo.cos(model.x[2, k]))
        if s == 1:  # y
            return model.x[1, k+1] == model.x[1, k] + dt * (model.x[4, k] * pyo.sin(model.x[2, k]))
        if s == 2:  # theta_t
            return model.x[2, k+1] == model.x[2, k] + dt * ((model.x[4, k] / L) * pyo.tan(model.x[5, k]))
        if s == 3:  # theta_l
            return model.x[3, k+1] == model.x[3, k] + dt * ((model.x[4, k] / d) * pyo.sin(model.x[2, k] - model.x[3, k]))
        if s == 4:  # v
            return model.x[4, k+1] == model.x[4, k] + dt * model.a[k]
        if s == 5:  # phi
            return model.x[5, k+1] == model.x[5, k] + dt * model.phi_dot[k]

    model.dyn = pyo.Constraint(model.S, model.K, rule=dyn_rule)

    # Jackknife constraint
    def jackknife_rule(model, t):
        return pyo.inequality(-max_jackknife, model.x[2, t] - model.x[3, t], max_jackknife)
    model.jackknife_cons = pyo.Constraint(model.T, rule=jackknife_rule)

    # Velocity & steering angle constraints
    model.vel_cons = pyo.Constraint(model.T, rule=lambda m, t: pyo.inequality(v_min, m.x[4, t], v_max))
    model.phi_cons = pyo.Constraint(model.T, rule=lambda m, t: pyo.inequality(phi_min, m.x[5, t], phi_max))

    # Obstacle avoidance constraints
    # For each obstacle and each time step, ensure minimum distance
    if obstacles:
        def obstacle_rule(model, o_idx, t):
            if o_idx >= len(obstacles):
                return pyo.Constraint.Skip
            
            obstacle = obstacles[o_idx]
            obs_x0 = obstacle["position"][0]
            obs_y0 = obstacle["position"][1]
            obs_vx = obstacle["velocity"][0]
            obs_vy = obstacle["velocity"][1]
            obs_radius = obstacle.get("radius", 0.5)
            
            # Predicted obstacle position at time step t
            obs_x_t = obs_x0 + obs_vx * t * dt
            obs_y_t = obs_y0 + obs_vy * t * dt
            
            # Truck position at time step t (hitch point)
            truck_x = model.x[0, t]
            truck_y = model.x[1, t]
            
            # Distance constraint: ||truck_pos - obstacle_pos|| >= safety_radius
            min_distance = obs_radius + truck_radius + safety_margin
            
            # Use squared distance to avoid square root (nonlinear constraint)
            # (x_truck - x_obs)^2 + (y_truck - y_obs)^2 >= min_distance^2
            dx = truck_x - obs_x_t
            dy = truck_y - obs_y_t
            return dx*dx + dy*dy >= min_distance * min_distance
        
        model.obstacle_cons = pyo.Constraint(model.O, model.T, rule=obstacle_rule)

    # Terminal constraints (optional, for stability and feasibility)
    # Note: With obstacles, terminal constraints may cause infeasibility
    # It's often better to use terminal cost only when obstacles are present
    use_terminal_constraint = params.get("use_terminal_constraint", False)
    terminal_tolerance = params.get("terminal_tolerance", 0.1)
    
    if use_terminal_constraint:
        terminal_type = params.get("terminal_constraint_type", "set")
        
        if terminal_type == "zero":
            # Zero terminal state constraint
            def terminal_zero_rule(model, i):
                return model.x[i, N] == target[i]
            model.terminal_cons = pyo.Constraint(range(4), rule=terminal_zero_rule)
            
        elif terminal_type == "set":
            # Terminal set constraint (recommended for obstacle avoidance)
            def terminal_set_rule(model):
                return sum((model.x[i, N] - target[i])**2 for i in range(4)) <= terminal_tolerance**2
            model.terminal_cons = pyo.Constraint(rule=terminal_set_rule)

    # Objective
    def obj_rule(model):
        final_err = sum((model.x[i, N] - target[i])**2 for i in range(4))
        control_effort = sum(model.a[k]**2 + model.phi_dot[k]**2 for k in model.K)
        return final_err + w_control * control_effort
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    return model


# --- MPC Loop with Obstacle Avoidance ---
def run_mpc_loop_obstacles(params):
    """
    Run MPC loop with obstacle avoidance.
    """
    MAX_STEPS = params.get("max_steps", 150)
    TARGET_TOL = params.get("target_tol", 0.5)

    solver = pyo.SolverFactory("ipopt")
    solver.options["max_iter"] = params.get("solver_max_iter", 500)
    solver.options["tol"] = params.get("solver_tol", 1e-6)
    solver.options["print_level"] = params.get("solver_print_level", 0)

    q_current = params["q0"].copy()
    N = params["N"]

    # initial guess for controls
    u_guess = np.zeros((2, N))
    u_guess[0, :] = params.get("a_init", 0.0)
    u_guess[1, :] = params.get("phi_dot_init", 0.0)

    q_history = [q_current.copy()]
    u_history = []
    step = 0
    
    # Track obstacle positions over time for animation
    obstacle_history = []
    if "obstacles" in params:
        # Store initial obstacle positions
        for obstacle in params["obstacles"]:
            obstacle_history.append({
                "initial_position": obstacle.get("initial_position", obstacle["position"].copy()),
                "velocity": obstacle["velocity"].copy(),
                "radius": obstacle.get("radius", 0.5),
                "positions": [obstacle.get("initial_position", obstacle["position"].copy())]
            })

    print("Starting MPC simulation with obstacle avoidance...")

    while np.linalg.norm(q_current[:2] - params["target"][:2]) > TARGET_TOL and step < MAX_STEPS:
        # Build model with updated obstacle positions (for moving obstacles)
        model = build_pyomo_model_obstacles(params)
        
        # Fix initial state
        for i in range(6):
            model.x[i, 0].fix(float(q_current[i]))

        set_initial_guess(model, u_guess)
        result = solver.solve(model, tee=False)
        term = result.solver.termination_condition
        
        # Handle solver termination more gracefully
        if term not in (pyo.TerminationCondition.optimal, pyo.TerminationCondition.locallyOptimal):
            if term == pyo.TerminationCondition.infeasible:
                print(f"Solver warning: {term} at step {step}")
                # Try to extract solution anyway - sometimes solver finds a close solution
                try:
                    x_opt, u_opt = extract_solution(model, params)
                    # Check if solution is reasonable
                    if np.any(np.isnan(x_opt)) or np.any(np.isnan(u_opt)):
                        print(f"  Solution contains NaN, stopping.")
                        break
                    # Check distance to target
                    dist_to_target = np.linalg.norm(q_current[:2] - params["target"][:2])
                    if dist_to_target < 2.0:  # Close enough, accept it
                        print(f"  Close to target ({dist_to_target:.2f}m), continuing...")
                    else:
                        print(f"  Using suboptimal solution (distance: {dist_to_target:.2f}m)")
                except Exception as e:
                    print(f"  Cannot extract solution: {e}")
                    break
            else:
                print(f"Solver warning: {term} at step {step}")
                try:
                    x_opt, u_opt = extract_solution(model, params)
                except:
                    break
        else:
            x_opt, u_opt = extract_solution(model, params)
        u_star = u_opt[:, 0]
        q_dot = truck_trailer_dynamics(q_current, u_star, params["L"], params["d"])
        q_next = q_current + params["dt"] * q_dot

        q_history.append(q_next.copy())
        u_history.append(u_star.copy())

        q_current = q_next.copy()
        step += 1

        u_guess = np.hstack([u_opt[:, 1:], u_opt[:, -1:]])
        
        # Update obstacle positions for next iteration (if obstacles are moving)
        if "obstacles" in params:
            for i, obstacle in enumerate(params["obstacles"]):
                obstacle["position"][0] += obstacle["velocity"][0] * params["dt"]
                obstacle["position"][1] += obstacle["velocity"][1] * params["dt"]
                # Track obstacle position for animation
                if i < len(obstacle_history):
                    obstacle_history[i]["positions"].append(obstacle["position"].copy())

    print(f"MPC finished in {step} steps. Final error: {np.linalg.norm(q_current[:2] - params['target'][:2]):.3f} m")
    
    # Store obstacle history in params for animation
    if obstacle_history:
        params["obstacle_history"] = obstacle_history
    
    return np.array(q_history), np.array(u_history)


In [ ]:
# Forward Navigation with Bicycle Obstacle Avoidance

# Define target
target_obstacle = np.array([15.0, 5.0, 0.0, 0.0])

bicycle_obstacle = {
    "position": [7.5, 8.0],   # Starting position at top, will cross truck path
    "velocity": [0.0, -2.5],  # Moving downward (negative y) at 2.5 m/s
    "radius": 0.2,            # Bicycle radius (m) - smaller than truck
    "initial_position": [7.5, 8.0],  # Store original for visualization
}

params_obstacle = {
    "L": 2.0, "d": 5.0,
    "target": target_obstacle,
    "q0": np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    "dt": 0.1, "N": 20,  # Longer horizon for obstacle avoidance
    "v_min": 0.0, "v_max": 3.0, 
    "phi_min": -1.5, "phi_max": 1.5,
    "a_min": -2.0, "a_max": 2.0, 
    "phi_dot_min": -1.0, "phi_dot_max": 1.0,
    "max_jackknife_angle": np.pi/2,
    "w_control": 0.01,
    "max_steps": 200, "target_tol": 0.5,
    "solver_max_iter": 1000, "solver_tol": 1e-6, "solver_print_level": 0,
    "a_init": 0.1, "phi_dot_init": 0.0,
    
    # Obstacle parameters
    "obstacles": [bicycle_obstacle],
    "safety_margin": 2.0,  # Additional safety margin (m)
    "truck_radius": 1.5,    # Approximate truck-trailer footprint radius (m)
}

# Run MPC with obstacle avoidance
q_hist_obs, u_hist_obs = run_mpc_loop_obstacles(params_obstacle)

# Visualize results
plot_trajectory(q_hist_obs, u_hist_obs, params_obstacle)

# Create enhanced visualization with obstacles
fig, ax = plt.subplots(figsize=(14, 10))
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)

# Plot trajectory
ax.plot(q_hist_obs[:, 0], q_hist_obs[:, 1], "b-", linewidth=2.5, label="Truck-Trailer Path", zorder=2)

# Plot start and target
ax.plot(params_obstacle["q0"][0], params_obstacle["q0"][1], "g*", ms=20, mew=2, label="Start", zorder=10)
ax.plot(target_obstacle[0], target_obstacle[1], "r*", ms=20, mew=2, label="Target", zorder=10)

# Plot obstacles at their initial and final positions
from matplotlib.patches import Circle
for i, obstacle in enumerate(params_obstacle["obstacles"]):
    # Get initial position (stored separately or calculate from current)
    if "initial_position" in obstacle:
        obs_init = obstacle["initial_position"].copy()
    else:
        # Calculate initial position from current position and velocity
        obs_init = [
            obstacle["position"][0] - obstacle["velocity"][0] * len(q_hist_obs) * params_obstacle["dt"],
            obstacle["position"][1] - obstacle["velocity"][1] * len(q_hist_obs) * params_obstacle["dt"]
        ]
    
    # Final position (current position after simulation)
    obs_final = obstacle["position"].copy()
    
    # Draw obstacle trajectory
    ax.plot([obs_init[0], obs_final[0]], [obs_init[1], obs_final[1]], 
            "r--", linewidth=2, alpha=0.5, label=f"Bicycle {i+1} Path", zorder=1)
    
    # Draw obstacle circles
    circle_init = Circle(obs_init, obstacle["radius"], facecolor='red', edgecolor='darkred', 
                        linewidth=2, alpha=0.6, zorder=5)
    circle_final = Circle(obs_final, obstacle["radius"], facecolor='darkred', edgecolor='black', 
                         linewidth=2, alpha=0.8, zorder=5)
    ax.add_patch(circle_init)
    ax.add_patch(circle_final)
    
    # Draw safety margin
    safety_radius = obstacle["radius"] + params_obstacle["truck_radius"] + params_obstacle["safety_margin"]
    safety_circle = Circle(obs_init, safety_radius, facecolor='none', edgecolor='orange', 
                          linewidth=1.5, linestyle='--', alpha=0.4, zorder=1, label=f"Safety Zone {i+1}")
    ax.add_patch(safety_circle)

ax.set_xlabel("x (m)", fontsize=12)
ax.set_ylabel("y (m)", fontsize=12)
ax.set_title("Truck-Trailer Navigation with Bicycle Obstacle Avoidance", fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
plt.tight_layout()
plt.show()

# Animate with moving obstacles
print("\nGenerating animation with moving obstacles...")
# Check if function exists, if not use regular animation (user needs to run cell 11 first)
if 'animate_trajectory_with_obstacles' in globals():
    anim_obs = animate_trajectory_with_obstacles(q_hist_obs, params_obstacle, interval=80, max_frames=500)
else:
    print("Warning: animate_trajectory_with_obstacles not found. Please run cell 11 first to define the function.")
    print("Using regular animation instead...")
    anim_obs = animate_trajectory(q_hist_obs, params_obstacle, interval=80, max_frames=500)
display(HTML(anim_obs.to_jshtml()))


### Reverse Point-to-Point Navigation

In [ ]:
params_reverse_point = {
    # Physical parameters
    "L": 2.0,           # truck wheelbase (m)
    "d": 5.0,           # hitch-to-trailer distance (m)

    # Target state [x, y, theta_t, theta_l]
    "target": np.array([np.random.uniform(-100, -80), np.random.uniform(-15, 15), 0.0, 0.0]),

    # Initial state [x, y, theta_t, theta_l, v, phi]
    "q0": np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),

    # MPC parameters
    "dt": 0.1,          # time step (s)
    "N": 200,            # prediction horizon

    # State bounds
    "v_min": -6.0,      # reverse only
    "v_max": 0.0,
    "phi_min": -1.5,
    "phi_max": 1.5,

    # Control bounds (acceleration & steering rate)
    "a_min": -2.0,
    "a_max": 2.0,
    "phi_dot_min": -0.5,
    "phi_dot_max": 0.5,

    # Constraints
    "max_jackknife_angle": np.pi/2,

    # Objective weights
    "w_control": 0.01,

    # Simulation parameters
    "max_steps": 1000,
    "target_tol": 0.5,

    # Solver settings
    "solver_max_iter": 500,
    "solver_tol": 1e-6,
    "solver_print_level": 0,

    # Initial guess
    "a_init": -0.1,      # small reverse acceleration
    "phi_dot_init": 0.0, # initial steering rate
}

# Run MPC
q_hist_rev, u_hist_rev = run_mpc_loop(params_reverse_point)

# Visualize results
plot_trajectory(q_hist_rev, u_hist_rev, params_reverse_point)

# Animate
anim_rev = animate_trajectory(q_hist_rev, params_reverse_point, interval=80)

# Display animation
display(HTML(anim_rev.to_jshtml()))
